In [29]:
import numpy as np
import scipy as sp

import ast
import os
import re

from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian
import quchem.Misc_functions.conversion_scripts as conv_scr 
from copy import deepcopy
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_reduced_H_matrix_SeqRot, SeqRot_linalg_Energy
from openfermion import qubit_operator_sparse

import pickle
import datetime

#######
import sys




In [9]:
working_dir = os.path.dirname(os.getcwd())
Analysis_dir = os.path.join(working_dir, 'Analysis')
full_H_results_dir = os.path.join(Analysis_dir, 'SeqRot_LCU_script_A_results')

In [10]:
###### IMPORT INITIAL RESULTS

## import SeqRot results
myriad_SeqRot_results = {}
for filename in os.listdir(full_H_results_dir):
    if (filename.endswith('.pickle') and filename.startswith('SeqRot_CS_VQE_exp')):
        file_path = os.path.join(full_H_results_dir, filename) 
        mol_name = filename[43:-8]
        with open(file_path,'rb') as infile:
            data = pickle.load(infile)
        myriad_SeqRot_results[mol_name] = data

In [50]:

######## take commandline arguement to run in parallel
AC_set_index  = 10 # minus one as array script idexes from 1
mol_key = 'C1-O1_STO-3G_singlet'

check_reduction_SeqRot = False

if mol_key not in myriad_SeqRot_results.keys():
    raise ValueError('molecule key not correct')

In [51]:
########
## import AC_sets results

AC_sets_dir_name = 'AC_sets_SeqRot'
AC_dir = os.path.join(working_dir, AC_sets_dir_name)
input_AC_file_path = os.path.join(AC_dir, mol_key + '.pickle') # AC of given molecule


with open(input_AC_file_path,'rb') as infile:
    all_anti_commuting_sets_SeqRot = pickle.load(infile)

anti_commuting_sets_SeqRot = all_anti_commuting_sets_SeqRot[AC_set_index]['AC_sets']
ground_state_ket = all_anti_commuting_sets_SeqRot[AC_set_index]['ground_state']

In [53]:
## Get Energy

if anti_commuting_sets_SeqRot:
    ### SeqRot
    all_zero_Ps_index_dict = {set_key: 0 for set_key in anti_commuting_sets_SeqRot}

    H_SeqRot_dict = myriad_SeqRot_results[mol_key][AC_set_index]['H']
    n_qubits = len(list(H_SeqRot_dict.keys())[0])

#     H_sparse = Get_reduced_H_matrix_SeqRot(anti_commuting_sets_SeqRot,
#                                      all_zero_Ps_index_dict,
#                                      n_qubits,
#                                      atol=1e-8,
#                                      rtol=1e-05,
#                                      check_reduction=check_reduction_SeqRot)
    
    
#     E_SeqRot = SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,
#                                  all_zero_Ps_index_dict,
#                                  n_qubits,
#                                  atol=1e-8,
#                                  rtol=1e-05,
#                                  check_reduction=check_reduction_SeqRot)

#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':E_SeqRot}
    
    
    ########
    
    
#     denisty_mat = np.outer(ground_state_ket, ground_state_ket)
#     E_SeqRot = np.trace(denisty_mat@H_sparse)

#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':E_SeqRot}
# else:
#     # only non-contextual problem
#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':myriad_SeqRot_results[mol_key][AC_set_index]['E']}    




In [32]:
%timeit SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

498 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%timeit SeqRot_linalg_Energy_FAST(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

253 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
E1= SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)
E2= SeqRot_linalg_Energy_FAST(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

In [64]:
from scipy.sparse import csr_matrix

In [75]:
def test1():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    H_single_terms = QubitOperator()
    gammal_Rdag_P_R_terms = QubitOperator()
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            H_single_terms += AC_set[0]
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)

            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            R_dag_P_R = Ps * R_S
            gammal_Rdag_P_R_terms += gamma_l*R_dag_P_R

    all_symbolic_ops = H_single_terms + gammal_Rdag_P_R_terms
    reduced_H_matrix = fast_qubit_operator_sparse(all_symbolic_ops, N_Qubits)

In [76]:
def test2():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    reduced_H_matrix = csr_matrix((2 ** N_Qubits, 2 ** N_Qubits), dtype=complex)
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            CiPi = AC_set[0]
            CiPi_matrix = fast_qubit_operator_sparse(CiPi, N_Qubits)
            reduced_H_matrix+=CiPi_matrix
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

            R_S_matrix = fast_qubit_operator_sparse(R_S, N_Qubits)
            P_S_matrix = fast_qubit_operator_sparse(Ps, N_Qubits)
            Rdag_P_R= R_S_matrix.conj().T @ P_S_matrix @ R_S_matrix
            reduced_H_matrix+=gamma_l*Rdag_P_R


In [79]:
def test3():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    reduced_H_matrix = csr_matrix((2 ** N_Qubits, 2 ** N_Qubits), dtype=complex)
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            CiPi = AC_set[0]
            CiPi_matrix = fast_qubit_operator_sparse(CiPi, N_Qubits)
            reduced_H_matrix+=CiPi_matrix
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)
            
            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            R_dag_P_R_matrix = fast_qubit_operator_sparse(R_dag_P_R, N_Qubits)
            reduced_H_matrix+=gamma_l*R_dag_P_R_matrix


In [77]:
%timeit test1()

1min 8s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%timeit test2()

26.3 s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
%timeit test3()

55.2 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%timeit R_S_matrix.conj().T

78.9 µs ± 333 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [83]:
%timeit fast_qubit_operator_sparse(hermitian_conjugated(R_S), N_Qubits)

1.17 ms ± 3.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
len(list(all_symbolic_ops)) * 0.5

442.0

In [16]:
# ####### SAVE OUTPUT details
# unique_file_time = datetime.datetime.now().strftime('%Y%b%d-%H%M%S%f')
# working_directory = os.getcwd()
# output_dir =os.path.join(working_directory, mol_key)

# # Create target Directory if it doesn't exist
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)


# # save file
# file_name1 = 'AC_set_and_Energy_output_set_key_{}.pickle'.format(AC_set_index)
# file_out1=os.path.join(output_dir, file_name1)

# ####### SAVE OUTPUT
# with open(file_out1, 'wb') as outfile:
#     pickle.dump(AC_set_and_Energy_output, outfile)


# print('pickle files dumped at: {}'.format(file_out1))

# print('end time: {}'.format(datetime.datetime.now().strftime('%Y%b%d-%H%M%S%f')))

In [44]:
from quchem.Misc_functions.Misc_functions import fast_qubit_operator_sparse
from openfermion import QubitOperator, hermitian_conjugated
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import  Get_Xsk_op_list, Get_Rsl_matrix_as_qubitops
from scipy.linalg import eigh
from scipy.sparse.linalg import eigsh

def SeqRot_linalg_Energy_FAST(anti_commuting_sets, S_key_dict, N_Qubits, atol=1e-8, rtol=1e-05, check_reduction=False):
    """
    Function giving ground state energy of Hamiltonian given as a dictionary of anti-commuting sets. Note this uses symbolic operators and only builds sparse matrix once.


    Args:
        anti_commuting_sets (dict): dictionary of int keys with list of anti commuting QubitOperators sets
        S_key_dict(dict): dictionary keys match that of anti_commuting_sets. Value gives index of P_s operator
        N_Qubits(int): number of qubits

    returns:
        FCI_Energy(float): Ground state energy

    """
    # TODO: could return reduced_H_matrix sparse matrix!


    H_single_terms = QubitOperator()
    gammal_Rdag_P_R_terms = QubitOperator()
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            H_single_terms += AC_set[0]
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)
            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            gammal_Rdag_P_R_terms += gamma_l*R_dag_P_R

    all_symbolic_ops = H_single_terms + gammal_Rdag_P_R_terms
    reduced_H_matrix = fast_qubit_operator_sparse(all_symbolic_ops, N_Qubits)
#     reduced_H_matrix = qubit_operator_sparse(all_symbolic_ops, n_qubits=N_Qubits)
    # eig_values, eig_vectors = sparse_eigs(reduced_H_matrix)
    if reduced_H_matrix.shape[0]<=64:
        eig_values, eig_vectors = eigh(reduced_H_matrix.todense()) # NOT sparse!
    else:
        eig_values, eig_vectors = eigsh(reduced_H_matrix, k=1, which='SA') # < solves eigenvalue problem for a complex Hermitian matrix.
    FCI_Energy = min(eig_values)
    return FCI_Energy